We'll write a simplifier a la Luckasz

In [2]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm


class Solver:
    def __init__(self, n_qubits=3, qlr=0.01, qepochs=200,verbose=0, g=1, J=0):

        """"solver with n**2 possible actions: n(n-1) CNOTS + n 1-qubit unitary"""
        self.n_qubits = n_qubits
        self.qubits = cirq.GridQubit.rect(1, n_qubits)
        self.lower_bound_Eg = -2*self.n_qubits
        
        self.qlr = qlr
        self.qepochs=qepochs
        self.verbose=verbose


        self.indexed_cnots = {}
        self.cnots_index = {}
        count = 0
        for control in range(self.n_qubits):
            for target in range(self.n_qubits):
                if control != target:
                    self.indexed_cnots[str(count)] = [control, target]
                    self.cnots_index[str([control,target])] = count
                    count += 1
        self.number_of_cnots = len(self.indexed_cnots)
        
        self.final_params = []
        self.parametrized_unitary = [cirq.rz, cirq.rx, cirq.rz]
        
        self.observable=self.ising_obs(g=g, J=J)
        
    def ising_obs(self, g=1, J=0):
        # g \sum_i Z_i - J \sum_{i} X_i X_{i+1}
        observable = [float(g)*cirq.Z.on(q) for q in self.qubits] 
        for q in range(len(self.qubits)):
            observable.append(float(J)*cirq.X.on(self.qubits[q])*cirq.X.on(self.qubits[(q+1)%len(self.qubits)]))
        self.ground_energy = -g*np.sum(np.sqrt([1+(J/4*g)**2 - (np.cos(q)*(J/2*g)) for q in range(self.n_qubits)]))
        return observable
        
    def index_meaning(self,index):
        if index<self.number_of_cnots:
            print("cnot: ",self.indexed_cnots[str(index)])
            return
        else:
            print("1-qubit unitary on: ",(index-self.number_of_cnots)%self.n_qubits)
            return

    def append_to_circuit(self, ind, circuit, params):
        """
        appends to circuit the index of the gate;
        and if one_hot_gate it implies a rotation,
        appends to params a symbol
        """

        if ind < self.number_of_cnots:
            control, target = self.indexed_cnots[str(ind)]
            circuit.append(cirq.CNOT.on(self.qubits[control], self.qubits[target]))
            return circuit, params
        else:
            qubit = self.qubits[(ind-self.number_of_cnots)%self.n_qubits]
            for par, gate in zip(range(3),self.parametrized_unitary):
                new_param = "th_"+str(len(params))
                params.append(new_param)
                circuit.append(gate(sympy.Symbol(new_param)).on(qubit))
            return circuit, params
        
    def give_circuit(self, lista,one_hot=False):
        circuit, symbols = [], []
        for k in lista:
            circuit, symbols = self.append_to_circuit(k,circuit,symbols)
        circuit = cirq.Circuit(circuit)
        return circuit, symbols

    def dressed_cnot(self,q1,q2):
        u1 = self.number_of_cnots + q1
        u2 = self.number_of_cnots + q2
        cnot = self.cnots_index[str([q1,q2])]
        u3 = self.number_of_cnots + q1
        u4 = self.number_of_cnots + q2
        return [u1,u2,cnot,u3,u4]
    
    def dressed_ansatz(self, layers=1):
        c=[]
        for layer in range(layers):
            qubits = list(range(self.n_qubits))
            qdeph = qubits[layers:]
            for q in qubits[:layers]:
                qdeph.append(q)
            for ind1, ind2 in zip(qubits,qdeph):
                for k in self.dressed_cnot(ind1,ind2):
                    c.append(k)
        return c


    def TFQ_model(self, symbols):
        circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
        output = tfq.layers.Expectation()(
                circuit_input,
                symbol_names=symbols,
                operators=tfq.convert_to_tensor([self.observable]),
                initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!
        model = tf.keras.Model(inputs=circuit_input, outputs=output)
        adam = tf.keras.optimizers.Adam(learning_rate=self.qlr)
        model.compile(optimizer=adam, loss='mse')
        return model


    def run_circuit(self, gates_index, sim_q_state=False):
        """
        takes as input vector with actions described as integer
        and outputsthe energy of that circuit (w.r.t self.observable)
        """
        ### create a vector with the gates on the corresponding qubit(s)
        circuit, symbols = self.give_circuit(gates_index)
        
        ### this is because each qubit should be "activated" in TFQ to do the optimization (if the observable has support on this qubit as well and you don't add I then error)
        effective_qubits = list(circuit.all_qubits())
        for k in self.qubits:
            if k not in effective_qubits:
                circuit.append(cirq.I.on(k))

        tfqcircuit = tfq.convert_to_tensor([circuit])
        if len(symbols) == 0:
            expval = tfq.layers.Expectation()(
                                            tfqcircuit,
                                            operators=tfq.convert_to_tensor([self.observable]))
            energy = np.float32(np.squeeze(tf.math.reduce_sum(expval, axis=-1, keepdims=True)))
            self.final_params = []

        else:
            model = self.TFQ_model(symbols)
            qoutput = tf.ones((1, 1))*self.lower_bound_Eg
            model.fit(x=tfqcircuit, y=qoutput, batch_size=1, epochs=self.qepochs, verbose=self.verbose)
            energy = np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))
            self.final_params = [np.squeeze(k.numpy()) for k in model.trainable_variables]

        #if sim_q_state:
            #simulator = cirq.Simulator()
            #result = simulator.simulate(circuit, qubit_order=self.qubits)
            #probs = np.abs(result.final_state)**2
            #return energy, probs
        return energy

In [3]:
sol = Solver()

In [4]:
sol.indexed_cnots

{'0': [0, 1], '1': [0, 2], '2': [1, 0], '3': [1, 2], '4': [2, 0], '5': [2, 1]}

In [14]:
indexed_circuit = [0,7,7,8]
sol.give_circuit(indexed_circuit)

((0, 0): ───@────────────────────────────────────────────────────────────────────────────
           │
(0, 1): ───X──────────Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_3)───Rx(th_4)───Rz(th_5)───

(0, 2): ───Rz(th_6)───Rx(th_7)───Rz(th_8)───────────────────────────────────────────────,
 ['th_0', 'th_1', 'th_2', 'th_3', 'th_4', 'th_5', 'th_6', 'th_7', 'th_8'])

In [71]:
indexed_circuit = [0,0,7,7,7]
sol.give_circuit(indexed_circuit)


((0, 0): ───@───@──────────────────────────────────────────────────────────────────────────────────────────────────────
           │   │
(0, 1): ───X───X───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_3)───Rx(th_4)───Rz(th_5)───Rz(th_6)───Rx(th_7)───Rz(th_8)───,
 ['th_0', 'th_1', 'th_2', 'th_3', 'th_4', 'th_5', 'th_6', 'th_7', 'th_8'])

In [74]:
#load circuit on each qubit
connections={str(q):["init"] for q in range(sol.n_qubits)}
simplified = []
flagged = [False]*len(indexed_circuit) #to check if you have seen the cnots
for q in range(sol.n_qubits):
    for nn,idq in enumerate(indexed_circuit):
        if idq<sol.number_of_cnots:
            control, target = sol.indexed_cnots[str(idq)]
            if q in [control, target] and not flagged[nn]: #if this particular CNOT i haven't append it to connections
                if not (connections[str(control)][-1] == "cnot_control" and connections[str(target)][-1] == "cnot_target"): #if the previous gate on each qubit is 
                    simplified.append(idq)
                connections[str(control)].append("cnot_control") 
                connections[str(target)].append("cnot_target")
                flagged[nn] = True #so you don't add the other

            #elif q in [control, target] and connections[str(control)][-1] == "cnot_control" and connections[str(target)][-1] == "cnot_target":
             #   :
        else:
            if idq%sol.n_qubits == q:
                if connections[str(q)][-1] != "u":
                    simplified.append(idq)
                connections[str(q)].append("u")
                

In [110]:

def simplify(sol,indexed_circuit):

    #load circuit on each qubit
    connections={str(q):["init"] for q in range(sol.n_qubits)}
    cnotsections={str(q):[] for q in range(sol.n_qubits)}

    simplified = []
    flagged = [False]*len(indexed_circuit) #to check if you have seen the cnots
    for q in range(sol.n_qubits):
        for nn,idq in enumerate(indexed_circuit):
            if idq<sol.number_of_cnots:
                control, target = sol.indexed_cnots[str(idq)]

                if q in [control, target] and not flagged[nn]: #if this particular CNOT i haven't append it to connections
                    #cnotsections[str(control)].append(idq)
                    #cnotsections[str(target)].append(idq)


                    if (connections[str(control)][-1] != "u") and (connections[str(control)][-1] != "init") and (connections[str(control)][-1] == connections[str(target)][-1]):
                        connections[str(control)].pop(-1)
                        connections[str(target)].pop(-1)
                        simplified()
                    else:
                        simplified.append(idq)
                        indexed_appendings[str()]
                    connections[str(control)].append(idq) 
                    connections[str(target)].append(idq)
                    flagged[nn] = True #so you don't add the other
                                #elif q in [control, target] and connections[str(control)][-1] == "cnot_control" and connections[str(target)][-1] == "cnot_target":
                 #   :
            else:
                if idq%sol.n_qubits == q:
                    if connections[str(q)][-1] != "u":
                        simplified.append(idq)
                    connections[str(q)].append("u")
    return simplified

In [ ]:
casa = [1,2,3,4,5]

In [116]:
indexed_circuit = [0,0,7,7,7]
simplified =simplify(sol,indexed_circuit)
sol.give_circuit(indexed_circuit)

((0, 0): ───@───@──────────────────────────────────────────────────────────────────────────────────────────────────────
           │   │
(0, 1): ───X───X───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_3)───Rx(th_4)───Rz(th_5)───Rz(th_6)───Rx(th_7)───Rz(th_8)───,
 ['th_0', 'th_1', 'th_2', 'th_3', 'th_4', 'th_5', 'th_6', 'th_7', 'th_8'])

In [117]:
sol.give_circuit(simplified)

((0, 0): ───@────────────────────────────────────
           │
(0, 1): ───X───Rz(th_0)───Rx(th_1)───Rz(th_2)───,
 ['th_0', 'th_1', 'th_2'])

In [108]:
sol.give_circuit(simplified)

((0, 0): ───@────────────────────────────────────
           │
(0, 1): ───X───Rz(th_0)───Rx(th_1)───Rz(th_2)───,
 ['th_0', 'th_1', 'th_2'])

In [87]:
casa

[1, 2]

((0, 0): ───@───@───@───@───@──────────────────────────────────────────────────────────────────────────────────────────────────────
           │   │   │   │   │
(0, 1): ───X───X───X───X───X───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_3)───Rx(th_4)───Rz(th_5)───Rz(th_6)───Rx(th_7)───Rz(th_8)───,
 ['th_0', 'th_1', 'th_2', 'th_3', 'th_4', 'th_5', 'th_6', 'th_7', 'th_8'])

In [81]:
sol.give_circuit(simplified)

((0, 0): ───@────────────────────────────────────
           │
(0, 1): ───X───Rz(th_0)───Rx(th_1)───Rz(th_2)───,
 ['th_0', 'th_1', 'th_2'])